In [1]:
import pandas as pd
import numpy as np
import os
import re, string, timeit
import nltk
import sys
sys.path.append('/workspace/Translation_lancet/')
from utils_lancet import *
from nltk.corpus import stopwords
from multiprocessing import Pool
import tqdm
import time
import re
import multiprocessing
import pyreadr

folder = "Multiglobal_result/"# "English_result/" # "EU_result/" # 

In [2]:
#with open("EU_result/corpus_tokens.txt", "r") as fp:
 #   corpus = json.load(fp)

In [2]:
df_kwic = pd.read_pickle(folder + 'df_kwic_health.pkl')

In [4]:
climate_dict = ["climate_change", "changing_climate", "climate_emergency", "climate_crisis", 
                "climate_decay", "global_warming","green_house", "temperature", "extreme_weather",
                "global_environmental_change", "climate_variability", "greenhouse", 
                "greenhouse-gas", "low_carbon", "ghge", "ghges", "renewable_energy", 
                "carbon_emission", "carbon_emissions", "carbon_dioxide", "carbon-dioxide",
                "co2_emission", "co2_emissions", "climate_pollutant", "climate_pollutants", 
                "decarbonization", "decarbonisation", "carbon_neutral", "carbon-neutral", 
                "carbon_neutrality", "climate_neutrality", "climate_action", "net-zero", "net_zero", 'carbonemission', "carbonemissions"]

compoundlist = ["air pollution", "mental disorder", "mental disorders", "climate change", "changing climate", "climate emergency", 
          "climate crisis", "climate decay", "global warming", "green house", "extreme weather", "global environmental change", 
          "climate variability",  "low carbon", "renewable energy", "carbon emission", "carbon emissions", "carbon dioxide", 
          "co2 emission", "co2 emissions", "climate pollutant", "climate pollutants", "carbon neutral", "carbon neutrality", 
          "climate neutrality", "climate action", "net zero", "covid 19", "corona virus", "sars cov 2"]


def kwic_df_search(row, search_list = climate_dict,):
    _, row = row
    corpus = " ".join ([str(row["left"]), str(row["right"])])
    kwic_health_climate = KWIC(search_list = search_list, corpus = corpus.split(), id_ = row.Id, compound_list = compoundlist, )
    kwic_df, _ = kwic_health_climate.kwic_search(window = 25,)
    kwic_df["health_key"] = row.key_word 
    kwic_df["old_index"] = row.token_index 
    return kwic_df
    
test = False
res = []

start = 0
cores = multiprocessing.cpu_count()
for df in np.array_split(df_kwic, 5):
    if __name__ == '__main__':
        with Pool(cores) as p:
            if test == True:
                df = df.iloc[:5]
            r = list(tqdm.tqdm(p.imap(kwic_df_search, df.iterrows()), total = len(df)))
            res.append(r)
            start=+1
            #break

100%|██████████| 173435/173435 [02:42<00:00, 1065.50it/s]


In [5]:
r_concat = [s for i in res for s in i]

In [6]:
res = pd.concat(r_concat)  

In [7]:
res.to_csv(folder + 'df_kwic_health_climate.csv')

In [8]:
intersection = pd.read_csv(folder + 'df_kwic_health_climate.csv')

In [9]:
# corss gender and covid

# covid dictionary
covid_dict = ["covid-19", "covid19", "covid_19", "corona", "corona_virus", "sars-cov-2", "covid", "coronavirus"]

# gender dictionary
gender_dict = ["gender", "male", "female", "man", "men", "woman", "women", "sex"]

In [14]:
def kwic_df_search(row, search_list = gender_dict,):
    _, row = row
    corpus = " ".join ([str(row["left"]), str(row["right"])])
    kwic_health_climate = KWIC(search_list = search_list, corpus = corpus.split(), id_ = row.Id, compound_list = compoundlist, )
    kwic_df, _ = kwic_health_climate.kwic_search(window = 25,)
    kwic_df["climate_key"] = row.key_word 
    kwic_df["health_key"] = row.health_key
    kwic_df["old_index"] = row.token_index 
    return kwic_df

test = False

cores = multiprocessing.cpu_count()
df = intersection[["left", "right", "Id", "old_index", "health_key", "key_word", "token_index"]]
if __name__ == '__main__':
    with Pool(cores) as p:
        if test == True:
            df = df.iloc[:500]
        r = list(tqdm.tqdm(p.imap(kwic_df_search, df.iterrows()), total = len(df)))
        res = pd.concat(r)  

100%|██████████| 29407/29407 [00:29<00:00, 988.95it/s] 


In [17]:
res.to_csv(folder + "gender.csv", index = False)
#res.to_csv(folder + "covid.csv", index = False)